In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 경로 설정
%cd /content/drive/MyDrive/성균관대학교 과제/3학년 2학기/캡스톤설계프로젝트

/content/drive/MyDrive/24-2 Capston


### 데이터 전처리

In [ ]:
import pandas as pd

df = pd.read_csv("final_dataset_1205.csv", encoding = 'utf-8-sig')

df.head()

In [ ]:
for i, dis in enumerate(df['disease']):
  df['disease'][i]=[dis[2:-2]]

for i, dis in enumerate(df['final_symptoms']):
  df['final_symptoms'][i]=[item.strip().strip("'\"") for item in dis[2:-2].split(",")]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['final_symptoms'][i]=[item.strip().strip("'\"") for item in dis[2:-2].split(",")]
<ipython-input-4-105c84c6577c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['final_symptoms'][i]=[item.strip().strip("'\"") for item in dis[2:-2].split(",")]
<ipython-input-4-105c84c6577c>:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on 

In [ ]:
# 데이터 유형 변환
df['cause'] = df['cause'].astype(int)
df['frequency'] = df['frequency'].astype(int)
df['duration'] = df['duration'].astype(int)
df['age'] = df['age'].astype(int)
df['social'] = df['social'].astype(int)
df['academic'] = df['academic'].astype(int)
df['occupational'] = df['occupational'].astype(int)
df['life-threatening'] = df['life-threatening'].astype(int)

In [ ]:
df.drop(columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.2'], inplace=True)

In [ ]:
# final_symptoms 컬럼의 빈 리스트에 '무증상' 추가
df['final_symptoms'] = df['final_symptoms'].apply(lambda x: ['무증상'] if not x else x)

###Dataload

In [ ]:
# 라이브러리 로드
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
import sklearn
import transformers
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader, Dataset, TensorDataset
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import openai
from openai import OpenAI
from torch.utils.data import TensorDataset
from transformers import BertTokenizer
from tqdm import tqdm
import pickle
import math
import copy
from torch.cuda.amp import autocast, GradScaler

In [ ]:
# 라이브러리 버전 확인
print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("PyTorch version:", torch.__version__) # BCEWithLogitsLoss도 torch 버전 확인
#print("Torch NN version:", nn.__version__)  # nn은 torch 모듈에 포함되므로 torch 버전만 확인하면 됨
#print("Torch Optim version:", optim.__version__)  # optim은 torch 모듈에 포함됨
#print("Torch Functional version:", F.__version__)  # F는 torch.nn.functional로, 별도 버전 정보는 없지만, torch 버전 참조
print("scikit-learn version:", sklearn.__version__)
print("Transformers version:", transformers.__version__)
print("OpenAI version:", openai.__version__)
#print("BertTokenizer version:", BertTokenizer.__version__)  # transformers 라이브러리의 일부분

Numpy version: 1.26.4
Pandas version: 2.2.2
PyTorch version: 2.5.1+cu121
scikit-learn version: 1.5.2
Transformers version: 4.46.3
OpenAI version: 1.54.5


In [ ]:
# gpu 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#post 토큰화

max_length=128
tokenizer=AutoTokenizer.from_pretrained('klue/bert-base')

all_filtered_sentences = [str(sentence) for sentence in df['filtered_sentences'].tolist()]


tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
text_inputs = tokenizer(
    all_filtered_sentences,
    padding='max_length',
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

# post input_ids 및 attention_mask 추출
text_input_ids = text_inputs['input_ids']  # shape: (num_rows, max_length)
text_attention_mask = text_inputs['attention_mask']  # shape: (num_rows, max_length)

print(text_input_ids.shape)
print(text_attention_mask.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

torch.Size([5926, 128])
torch.Size([5926, 128])


In [ ]:
#evidence 관련 함수
# 토크나이징 (input_ids 및 attention_mask 생성)
def create_attention_mask(input_ids):
    return [1] * len(input_ids) + [0] * (max_length - len(input_ids))

# factor 증거(evidence) 결합
def combine_factor_inputs(evidence_input_ids_list, evidence_attention_mask_list):
    factor_input_ids, factor_attention_mask = [], []
    total_length = 0
    for evidence_input_ids, evidence_attention_mask in zip(evidence_input_ids_list, evidence_attention_mask_list):
        truncated_input_ids = evidence_input_ids[:max_length]
        truncated_attention_mask = evidence_attention_mask[:max_length]
        remaining_length = max_length - total_length
        if remaining_length <= 0:
            break
        factor_input_ids.extend(truncated_input_ids[:remaining_length])
        factor_attention_mask.extend(truncated_attention_mask[:remaining_length])
        total_length += len(truncated_input_ids)
    # 길이를 max_length로 제한
    return factor_input_ids[:max_length], factor_attention_mask[:max_length]

In [ ]:
#evidence
all_factor_input_ids = []
all_factor_attention_mask = []

for _, row in df.iterrows():  # iterrows()로 각 행 반복
    cause_evidence = row['cause_evidence']
    freq_evidence = row['freq_evidence']
    duration_evidence = row['duration_evidence']
    age_evidence = row['age_evidence']
    social_evidence = row['social_evidence']
    academic_evidence = row['academic_evidence']
    occupational_evidence = row['occupational_evidence']
    life_threatening_evidence = row['life-threatening_evidence']

    # evidence들을 각 토큰화된 input_ids로 변환
    evidence_fields = [
        cause_evidence, freq_evidence, duration_evidence, age_evidence,
        social_evidence, academic_evidence, occupational_evidence, life_threatening_evidence
    ]

    evidence_input_ids_list = [
        tokenizer.encode(evidence, add_special_tokens=True, truncation=True,
                         padding='max_length', max_length=max_length)
        for evidence in evidence_fields
    ]

    # attention_mask 생성
    evidence_attention_mask_list = [create_attention_mask(input_ids) for input_ids in evidence_input_ids_list]

    # factor 증거(evidence) 결합
    factor_input_ids, factor_attention_mask = combine_factor_inputs(
        evidence_input_ids_list, evidence_attention_mask_list
    )

    # 누적
    all_factor_input_ids.append(factor_input_ids)
    all_factor_attention_mask.append(factor_attention_mask)

# 최종 텐서로 변환
factor_input_ids = torch.tensor(all_factor_input_ids)
factor_attention_mask = torch.tensor(all_factor_attention_mask)

print(factor_input_ids.shape)  # (5926, 128)
print(factor_attention_mask.shape)  # (5926, 128)

torch.Size([5926, 128])
torch.Size([5926, 128])


In [ ]:
# 증상 데이터
'''
final_symptoms = [str(symp) for symp in df['final_symptoms'].tolist()]
symptom_inputs = tokenizer(
    final_symptoms,
    padding='max_length',
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)
symptom_input_ids = symptom_inputs['input_ids']
symptom_attention_mask = symptom_inputs['attention_mask']

print(symptom_input_ids.shape)
print(symptom_attention_mask.shape)
'''
# 고유한 48개의 증상 리스트 (모델의 타겟 클래스)
target_classes = ['무증상', '짜증이 잘 남', '두려움과 공포', '호흡곤란', '폭식', '답답함', '초조하고 안절부절 못함', '언어 능력 감소',
                  '피해망상', '불면', '성에 대한 과도한 관심', '우울함', '강박사고', '음식 섭취 제한', '자살사고', '다리 불편감', '기억력 저하',
                  '체중과 체형에 대한 집착', '무기력', '주의산만', '긴장', '감정 기복', '환각', '수면 무호흡증', '환청', '수면 과다', '충동성',
                  '피로', '메스꺼움', '어지러움', '눈물 흘림', '예민함', '극단적인 생각', '반복행동', '구토와 약물', '수면문제', '불안함',
                  '자극적인 행위에 몰입', '통제력 부족', '분노 조절 문제', '발작', '두통', '가슴 두근거림', '장시간 지속되는 이유 없는 불안감',
                  '집중력 저하', '일상생활 능력의 손상', '식욕감소', '걱정이 많아짐']

mlb_symp = MultiLabelBinarizer(classes=target_classes)  # MultiLabelBinarizer로 48개의 고유 증상 기준으로 변환
symptom_labels = mlb_symp.fit_transform(df['final_symptoms'])  # 레이블

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [''] will be ignored
  warnings.warn(


In [ ]:
#context categorical

# 모든 factors_tensor를 담을 리스트 생성
all_factors_tensor = []

for i, row in df.iterrows():  # 각 행을 순회
    # 각 특성별 텐서 정의
    factors = {
        'cause': row['cause'],
        'frequency': row['frequency'],
        'duration': row['duration'],
        'age': row['age'],
        'social': row['social'],
        'academic': row['academic'],
        'occupational': row['occupational'],
        'life-threatening': row['life-threatening']
    }

    # 특성 텐서들을 하나로 합침
    factors_tensor = torch.tensor([factors[key] for key in factors], dtype=torch.float32)

    # 결과를 리스트에 추가
    all_factors_tensor.append(factors_tensor)

# 리스트를 텐서로 변환
factors_tensor = torch.stack(all_factors_tensor)

# 최종 텐서 크기 확인
print(factors_tensor.shape)

torch.Size([5926, 8])


In [ ]:
# 16개의 고유 질병 클래스 정의
disease_classes = ["수면장애", "우울증", "섭식장애", "스트레스", "불안장애", "ADHD", "Non-disease", "공황장애",
                   "강박장애", "사회 불안장애", "중독(의존)", "하지 불안 증후군", "치매", "틱장애", "조현병", "양극성장애"]

# MultiLabelBinarizer로 레이블 이진화
mlb_disease = MultiLabelBinarizer(classes=disease_classes)
disease_labels = mlb_disease.fit_transform(df['disease'])  # 각 샘플에 대해 레이블 이진화
disease_labels.shape

(5926, 16)

## Modeling

### Symp context

In [ ]:
# 모델 정의
class SympWithFactor(nn.Module):
    def __init__(self, input_dim, num_factors, filter_num=64, filter_sizes=[1], dropout=0.2, num_labels=16, max_pooling_k=1):
        super(SympWithFactor, self).__init__()
        self.filter_num = filter_num
        self.filter_sizes = filter_sizes
        self.hidden_size = len(filter_sizes) * filter_num
        self.max_pooling_k = max_pooling_k
        self.convs = nn.ModuleList([nn.Conv1d(16, filter_num, size) for size in filter_sizes]) #num_factors * 2->16
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.hidden_size, num_labels)

    def forward(self, symptoms, factors):
        # symptoms를 축소하여 factors와 동일한 크기로 조정
        symptoms_reduced = self.kmax_pooling(symptoms.unsqueeze(1), k=factors.size(1)).squeeze(1)  # [batch_size, num_factors]

        # 축소된 symptoms와 factors를 결합
        input_seqs = torch.cat([symptoms_reduced, factors], dim=1).unsqueeze(1)  # [batch_size, 1, num_factors * 2]

        # Conv1d에 맞게 차원 전환
        input_seqs = input_seqs.transpose(1, 2)  # [batch_size, num_factors * 2, 1]
        x = [F.relu(conv(input_seqs)) for conv in self.convs]
        x = [self.kmax_pooling(item, self.max_pooling_k).mean(2) for item in x]
        x = torch.cat(x, 1)
        x = self.dropout(x)
        logits = self.fc(x)
        return logits

    def kmax_pooling(self, x, k):
        return x.sort(dim=2, descending=True)[0][:, :, :k]

In [ ]:
#symptoms_tensor, factors, labels_tensor

# 데이터셋 생성
dataset = TensorDataset(torch.tensor(symptom_labels, dtype=torch.float32), factors_tensor, torch.tensor(disease_labels, dtype=torch.float32))

# 훈련 세트와 검증 세트 분할
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# DataLoader로 배치 분할
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 클래스 가중치 계산
from sklearn.utils.class_weight import compute_class_weight
import torch

# 멀티라벨 데이터를 1차원으로 변환
# train_loader에서 모든 레이블 수집 (batch 차원 합침)
all_labels = torch.cat([labels for _, _, labels in train_loader], dim=0).cpu().numpy()  # (num_samples, num_labels)

# 각 클래스에 대해 가중치 계산
class_weights = []
for i in range(all_labels.shape[1]):  # 16개 클래스에 대해 반복
    class_weight = compute_class_weight(
        class_weight='balanced',
        classes=np.array([0, 1]),  # 클래스는 0과 1 (binary classification)
        y=all_labels[:, i]
    )
    class_weights.append(class_weight[1])  # 1 클래스의 가중치만 저장

# 텐서로 변환
class_weights = torch.tensor(class_weights).to(device)

# 결과 출력
print("Class Weights:", class_weights)

Class Weights: tensor([ 3.0649,  3.2598,  4.7447,  5.5207,  5.7967,  7.2068,  4.9934,  9.1632,
         9.8033, 12.5188, 19.4635, 22.4076, 30.3011, 59.2556, 63.4881, 91.9483],
       device='cuda:0', dtype=torch.float64)


In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 모델 및 옵티마이저 설정
SYMP_context = SympWithFactor(
    input_dim=8,  # 입력 벡터 크기
    num_factors=4,                         # factor 수
    filter_num=16,                         # 필터 개수
    filter_sizes=[1],                      # 필터 크기
    dropout=0.2,                           # 드롭아웃 비율
    num_labels=16           # 다중 레이블 클래스 개수
)
SYMP_context.to(device)

# 옵티마이저 및 손실 함수 설정
optimizer = torch.optim.AdamW(SYMP_context.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights).to(device)  # 다중 라벨 분류를 위한 손실 함수

# 스케줄러 설정
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

# 조기 종료 설정
min_val_loss = float('inf')
steps_since_improvement = 0
early_stop_step = 5

# 훈련 루프
for epoch in range(10):  # 최대 에포크 수
    SYMP_context.train()
    total_loss = 0

    for symptoms, factors, labels in train_loader:
        symptoms, factors, labels = symptoms.to(device), factors.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = SYMP_context(symptoms, factors)  # 모델에 symptoms와 factors를 전달
        loss = criterion(logits, labels.float())  # BCEWithLogitsLoss expects float labels
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # 학습 손실 로그 출력
    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}')

    # ===== 검증 단계 =====
    SYMP_context.eval()
    total_val_loss = 0

    with torch.no_grad():
        for symptoms, factors, labels in val_loader:  # 검증 데이터 로더 사용
            symptoms, factors, labels = symptoms.to(device), factors.to(device), labels.to(device)
            logits = SYMP_context(symptoms, factors)
            loss = criterion(logits, labels.float())
            total_val_loss += loss.item()

    # 검증 손실 로그 출력
    avg_val_loss = total_val_loss / len(val_loader)
    print(f'Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}')

    # ===== 스케줄러 업데이트 =====
    scheduler.step(avg_val_loss)

    # ===== 조기 종료 로직 =====
    if avg_val_loss < min_val_loss:
        min_val_loss = avg_val_loss
        steps_since_improvement = 0
        print(f"Validation loss improved to {min_val_loss:.4f}.")
    else:
        steps_since_improvement += 1
        print(f"No improvement in validation loss for {steps_since_improvement} steps.")

    if steps_since_improvement >= early_stop_step:
        print(f"Early stopping triggered at epoch {epoch+1}.")
        break

Epoch 1, Train Loss: 0.9740
Epoch 1, Validation Loss: 0.9935
Validation loss improved to 0.9935.
Epoch 2, Train Loss: 0.9402
Epoch 2, Validation Loss: 0.9721
Validation loss improved to 0.9721.
Epoch 3, Train Loss: 0.9232
Epoch 3, Validation Loss: 0.9483
Validation loss improved to 0.9483.
Epoch 4, Train Loss: 0.9051
Epoch 4, Validation Loss: 0.9346
Validation loss improved to 0.9346.
Epoch 5, Train Loss: 0.8937
Epoch 5, Validation Loss: 0.9202
Validation loss improved to 0.9202.
Epoch 6, Train Loss: 0.8814
Epoch 6, Validation Loss: 0.9095
Validation loss improved to 0.9095.
Epoch 7, Train Loss: 0.8691
Epoch 7, Validation Loss: 0.8938
Validation loss improved to 0.8938.
Epoch 8, Train Loss: 0.8548
Epoch 8, Validation Loss: 0.8854
Validation loss improved to 0.8854.
Epoch 9, Train Loss: 0.8460
Epoch 9, Validation Loss: 0.8744
Validation loss improved to 0.8744.
Epoch 10, Train Loss: 0.8366
Epoch 10, Validation Loss: 0.8681
Validation loss improved to 0.8681.


In [ ]:
# 각 타겟 threshold 출력
probabilities = torch.sigmoid(logits)
print(probabilities)
print(probabilities.mean())

tensor([[0.2375, 0.5120, 0.4845, 0.3649, 0.4289, 0.1303, 0.1521, 0.4164, 0.3445,
         0.3188, 0.2227, 0.2493, 0.2441, 0.2420, 0.3505, 0.3776],
        [0.3754, 0.4195, 0.4445, 0.3397, 0.3573, 0.1332, 0.2009, 0.4183, 0.2681,
         0.1979, 0.2007, 0.3384, 0.2652, 0.2036, 0.3051, 0.3095],
        [0.3118, 0.4809, 0.4800, 0.3438, 0.3926, 0.1271, 0.1311, 0.5034, 0.2426,
         0.1742, 0.1667, 0.2664, 0.1916, 0.1109, 0.2988, 0.3399],
        [0.4338, 0.2837, 0.2728, 0.3432, 0.3470, 0.2448, 0.3937, 0.3469, 0.3012,
         0.3066, 0.2542, 0.3802, 0.3724, 0.3008, 0.3903, 0.2926],
        [0.4274, 0.2271, 0.2305, 0.3224, 0.2794, 0.3029, 0.5292, 0.2748, 0.2964,
         0.3407, 0.2939, 0.3572, 0.3997, 0.4322, 0.3900, 0.2689],
        [0.2278, 0.5343, 0.5126, 0.3303, 0.4078, 0.0945, 0.0914, 0.5242, 0.2174,
         0.1756, 0.1356, 0.1954, 0.1269, 0.0809, 0.2762, 0.3420],
        [0.5197, 0.2969, 0.3712, 0.3646, 0.4096, 0.1537, 0.1637, 0.3311, 0.4041,
         0.2940, 0.2820, 0.5046, 0.38

In [ ]:
# 모델 검증
average_threshold = probabilities.mean()

SYMP_context.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for symptoms, factors, disease_labels in val_loader:
        # 데이터가 텐서인지 확인 및 변환
        symptoms, factors, disease_labels = (
            torch.tensor(symptoms, dtype=torch.float32).to(device)
            if isinstance(symptoms, list) else symptoms.to(device),
            torch.tensor(factors, dtype=torch.float32).to(device)
            if isinstance(factors, list) else factors.to(device),
            torch.tensor(disease_labels, dtype=torch.float32).to(device)
            if isinstance(disease_labels, list) else disease_labels.to(device),
        )

        # 모델 예측
        logits = SYMP_context(symptoms, factors)
        probabilities = torch.sigmoid(logits)
        predicted_labels = (probabilities > average_threshold).int()
        predictions.append(predicted_labels)
        true_labels.append(disease_labels)


predictions = torch.cat(predictions).cpu()
true_labels = torch.cat(true_labels).cpu()

precision = precision_score(true_labels, predictions, average='macro', zero_division=1)
recall = recall_score(true_labels, predictions, average='macro', zero_division=1)
f1 = f1_score(true_labels, predictions, average='macro', zero_division=1)

print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

#이전성능: Precision: 0.0950, Recall: 0.7858, F1 Score: 0.1571

Precision: 0.1104, Recall: 0.7929, F1 Score: 0.1765


In [ ]:
# 저장 파일 경로
pkl_save_path = "model/symp_context.pkl"
pth_save_path = "model/symp_context.pth"

# 전체 모델 저장 (pkl 형식)
torch.save(SYMP_context, pkl_save_path)
print(f"Entire model saved to {pkl_save_path}!!!")

# 상태 사전(state_dict) 저장 (pth 형식)
torch.save(SYMP_context.state_dict(), pth_save_path)
print(f"Model state_dict saved to {pth_save_path}!!!")

Entire model saved to symp_context_최종.pkl!!!
Model state_dict saved to symp_context_최종.pth!!!
